Démonstration

In [13]:
from threading import Thread 
import time
def simple_hello_world():
    time.sleep(5)
    print("hello world")
    
t1 = Thread(target=simple_hello_world)
t1.start()

hello world


In [11]:
from threading import Thread 

def simple_hello_world():
    print("hello world")
    
t1 = Thread(target=simple_hello_world)
t1.start()
try:
    t1.start()
except RuntimeError as e:
    print(f"\n{e}")

hello world
threads can only be started once



In [15]:
def info(thread):
    print(thread.name)
    print(thread.ident)
    print(thread.is_alive())

t1 = Thread(target=simple_hello_world)
t1.start()
info(t1)

t2 = Thread(target=simple_hello_world)
t2.start()
info(t2)

Thread-18 (simple_hello_world)
38628
True
Thread-19 (simple_hello_world)
10044
True
hello world
hello world


In [17]:
def print_to_console(caractere):
    for i in range(0, 10):
        print(caractere, end="", flush=True)
        time.sleep(2)
thread_asterix = Thread(target=print_to_console, args="*")
thread_point = Thread(target=print_to_console, args=".")
thread_asterix.start()
time.sleep(10)
thread_point.start()

*****.*.*.*.*.*.....

In [20]:
def print_to_console(caractere):
    for i in range(0, 10):
        print(caractere, end="", flush=True)
        time.sleep(1)
thread_asterix = Thread(target=print_to_console, args="*")
thread_point = Thread(target=print_to_console, args=".")
thread_asterix.start()

thread_asterix.join()
thread_point.start()

**********..........

In [24]:
class CustomThread(Thread):
    def __init__(self, caractere = "", name = None, daemon = None, target = None):
        super().__init__(name = name, daemon = daemon, target = target)
        self.name = name
        self.caractere = caractere
        
    def run(self):
        print(self.name)
        for i in range(0, 10):
            print(self.caractere, end="", flush=True)
            time.sleep(1)
thread_asterix = CustomThread("*", name = "Asterix")
thread_point = CustomThread(".", name = "Point")
thread_asterix.start()
thread_point.start()
    

AsterixPoint
.
*.**..**..**.*..**.

Démonstration du besoin de mémoire local par thread

In [35]:
import threading
import time
import random
from threading import Thread

class DataPartagee:
    value = None
    
def show_value(data):
    try:
        val = data.value
    except AttributeError:
        print(threading.current_thread().name, " Value n'est pas assignée")
    else:
        print(threading.current_thread().name, " Value vaut ", val)

def worker(data):
    #print(f"{threading.current_thread().name} globals : {list(globals().keys())}")
    show_value(data)
    data.value = random.randint(0, 10)
    show_value(data)
    time.sleep(2)
    show_value(data)

print(threading.current_thread(), "- Starting ")

data = DataPartagee()

for i in range(2):
    t = Thread(name = "T" +str(i+1), target=worker, args=(data,))
    t.start()    

print(threading.current_thread(), " -  Se Termine")

<_MainThread(MainThread, started 10056)> - Starting 
T1  Value vaut  None
T1  Value vaut  8
T2  Value vaut  8
T2  Value vaut  0
<_MainThread(MainThread, started 10056)>  -  Se Termine
T1  Value vaut  0
T2  Value vaut  0


Démonstration de la mémoire local par thread


In [39]:
import threading
import time
import random
from threading import Thread

class DataPartagee:
    value = None
    
def show_value(data):
    try:
        val = data.value
    except AttributeError:
        print(threading.current_thread().name, " Value n'est pas assignée")
    else:
        print(threading.current_thread().name, " Value vaut ", val)

def worker(data):
    #print(f"{threading.current_thread().name} globals : {list(globals().keys())}")
    show_value(data)
    data.value = random.randint(0, 10)
    show_value(data)
    time.sleep(2)
    show_value(data)

print(threading.current_thread(), "- Starting ")

local_data = threading.local()

for i in range(2):
    t = Thread(name = "T" +str(i+1), target=worker, args=(local_data,))
    t.start()

worker(local_data)

print(threading.current_thread(), " -  Se Termine")

<_MainThread(MainThread, started 10056)> - Starting 
T1  Value n'est pas assignée
T1  Value vaut  9
T2  Value n'est pas assignée
T2  Value vaut  6
MainThread  Value n'est pas assignée
MainThread  Value vaut  3
T1  Value vaut  9
MainThread  Value vaut  3
<_MainThread(MainThread, started 10056)>  -  Se Termine
T2  Value vaut  6


Démonstration que la mémoire local est vraiment par thread ! Du coup, il est "difficile" de partager de la donnée

In [2]:
import threading
from threading import Thread
class CustomString:
    value = " Je suis le message "
    
class StringProcessor(Thread): 
    def __init__(self, message, target = None, name = None, daemon = False):
        super().__init__(target = target, name = name, daemon = daemon)
        #attention msg est une reference
        self.message = message
        #est setter que dans le MainThread attention ne fonctionne pas
        local_data.value = message
        
    def run(self):
        try :
            self.message.value = self.message.value * 2
            print(threading.current_thread().name, "Le résultat est : " + self.message.value, ".")
        except AttributeError:
            print(threading.current_thread().name, "self.message.value n'est pas assignée")
        try :
            print(local_data.value)
        except AttributeError:
            print(threading.current_thread().name, " lcoal_data.value n'est pas assignée")
        
local_data = threading.local()
customstring = CustomString
for i in range(2):
    t = StringProcessor(customstring)
    t.start()


Thread-5 Le résultat est :  Je suis le message  Je suis le message  .
Thread-5  lcoal_data.value n'est pas assignée
Thread-6 Le résultat est :  Je suis le message  Je suis le message  Je suis le message  Je suis le message  .
Thread-6  lcoal_data.value n'est pas assignée


Démonstration qu'il faut passer par les argumetns pour puvoir copier de la mémoire global
auc local par thread pour rendre des obkjcts accesible mais unique par thread 


In [12]:
import threading
from threading import Thread
class CustomString:
    value = " Je suis le message "
        
def worker(customstring):
    try :
        local_data.customstring = customstring
        local_data.customstring.value = local_data.customstring.value * 2
        print(threading.current_thread().name, "Le résultat est : " + local_data.customstring.value, ".")
    except AttributeError:
        print(threading.current_thread().name, "self.message.value n'est pas assignée")
        
local_data = threading.local()
customstring = CustomString
for i in range(2):
    t = Thread(args=(customstring,), target=worker)
    t.start()


Thread-22 (worker) Le résultat est :  Je suis le message  Je suis le message  .
Thread-23 (worker) Le résultat est :  Je suis le message  Je suis le message  Je suis le message  Je suis le message  .
